In [4]:

import os
import argparse
import json
import glob
import nibabel as nb
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as pltpy
import seaborn
from brainspace.datasets import load_mask
from brainspace.plotting import plot_hemispheres
from brainspace.mesh.mesh_io import read_surface
from brainspace.mesh.mesh_operations import combine_surfaces
from brainspace.utils.parcellation import map_to_labels
from brainspace.vtk_interface import wrap_vtk, serial_connect
from vtk import vtkPolyDataNormals


In [5]:
import numpy as np
import nibabel as nb
#from surfplot import Plot
from brainspace.mesh import mesh_io

# Assuming read_surface is a function from brainspace.mesh.mesh_io
read_surface = mesh_io.read_surface

surf_lh = read_surface('/NAS/fatmike/Protocoles_3T/Neurosclerodermie/data/bids/derivatives/fastsurfer/sub-P0101_ses-1/surf/lh.inflated', itype='fs')
surf_rh = read_surface('/NAS/fatmike/Protocoles_3T/Neurosclerodermie/data/bids/derivatives/fastsurfer/sub-P0101_ses-1/surf/rh.inflated', itype='fs')


fileL = "/NAS/fatmike/Protocoles_3T/Neurosclerodermie/data/bids/derivatives/fastsurfer/sub-P0101_ses-1/label/lh.aparc_mics.annot"
fileR = "/NAS/fatmike/Protocoles_3T/Neurosclerodermie/data/bids/derivatives/fastsurfer/sub-P0101_ses-1/label/rh.aparc_mics.annot"
figPath = '/tmp/16428_micapipe_QC_P0101/sub-P0101_ses-1_atlas-aparc_desc-surf.png'
label = np.concatenate((nb.freesurfer.read_annot(fileL)[0], nb.freesurfer.read_annot(fileR)[0]), axis=0)


In [14]:
th = np.concatenate((nb.freesurfer.read_morph_data('/NAS/fatmike/Protocoles_3T/Neurosclerodermie/data/bids/derivatives/fastsurfer/sub-P0101_ses-1/surf/lh.thickness'), nb.freesurfer.read_morph_data('/NAS/fatmike/Protocoles_3T/Neurosclerodermie/data/bids/derivatives/fastsurfer/sub-P0101_ses-1/surf/rh.thickness')), axis=0)

FileNotFoundError: [Errno 2] No such file or directory: '/NAS/fatmike/Protocoles_3T/Neurosclerodermie/data/bids/derivatives/fastsurfer/sub-P0101_ses-1/surf/surf/rh.thickness'

In [7]:
def cmap_gradient(N, base_cmaps=['inferno', 'Dark2', 'Set1', 'Set2']):
    """
    Creates a gradient color map of a defined lenght.

    Parameters
    ----------
    N     : int
         Number of colors to extract from each of the base_cmaps below.
    cmaps : list
        List of color map(s) to merge.

    Returns
    -------
    cmap : numpy ndarray
        Surfaces for left and right hemispheres. If ``join == True``, one
        surface with both hemispheres.
    """
    # number of colors
    N = 75

    # we go from 0.2 to 0.8 below to avoid having several whites and blacks in the resulting cmaps
    colors = np.concatenate([pltpy.get_cmap(name)(np.linspace(0,1,N)) for name in base_cmaps])
    cmap = plt.colors.ListedColormap(colors.tolist())
    return cmap


In [13]:

plot_hemispheres(surf_lh, surf_rh, array_name=label, size=(900, 250), zoom=1.25, embed_nb=True, interactive=False, share='both',
                             nan_color=(0, 0, 0, 1),cmap=cmap_gradient(len(np.unique(label)), ['inferno', 'hsv', 'hsv', 'tab20b']), transparent_bg=False,
                             screenshot = True, offscreen=False, filename = figPath)


TypeError: unhashable type: 'ListedColormap'